# 🤖 AI, CA3, Machine Learning 📚  

* **Name** : Mohammad Sadra 🖊  
* **Last Name** :  Abbasi 📝  
* **SID** : 810101469 🆔

## 🛠 From Scratch  
In this subsection, you should implement the Decision Tree algorithm 🌳 from scratch 🛠 and compare its performance with the library implementation 📚.

In [ ]:
import numpy as np
import pandas as pd

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class MyDecisionTree:
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        X_values = X.values if isinstance(X, pd.DataFrame) else X
        y_values = y.values if isinstance(y, pd.Series) else y
        
        self.root = self._build_tree(X_values, y_values, depth=0)

    def predict(self, X):
        X_values = X.values if isinstance(X, pd.DataFrame) else X
        y_pred = [self._traverse_tree(x, self.root) for x in X_values]
        return np.array(y_pred)

    def _most_common_label(self, y):
        if len(y) == 0: return 0
        return np.bincount(y).argmax()
    
    def _entropy(self, y):
        class_labels, counts = np.unique(y, return_counts=True)
        probabilities = counts / counts.sum()
        entropy = -np.sum(probabilities * np.log2(probabilities + 1e-9))
        return entropy
    
    def _information_gain(self, X, y, feature, threshold):
        parent_entropy = self._entropy(y)

        left_indices = X[:, feature] <= threshold
        right_indices = X[:, feature] > threshold

        if len(y[left_indices]) == 0 or len(y[right_indices]) == 0:
            return 0

        n = len(y)
        n_left, n_right = len(y[left_indices]), len(y[right_indices])

        e_left = self._entropy(y[left_indices])
        e_right = self._entropy(y[right_indices])

        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right

        ig = parent_entropy - child_entropy
        return ig

    def _best_split(self, X, y, n_features):
        best_feature, best_threshold = None, None
        best_gain = -1

        for feature in range(n_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                gain = self._information_gain(X, y, feature, threshold)

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        
        return best_feature, best_threshold
    
    def _build_tree(self, X, y, depth):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        if (self.max_depth is not None and depth >= self.max_depth) or n_labels == 1 or n_samples < 2:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        best_feature, best_threshold = self._best_split(X, y, n_features)
        
        if best_feature is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold
        
        if len(y[left_indices]) == 0 or len(y[right_indices]) == 0:
             return Node(value=self._most_common_label(y))

        left_child = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_child = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return Node(feature=best_feature, threshold=best_threshold, left=left_child, right=right_child)
    
    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        else:
            return self._traverse_tree(x, node.right)
        

    # Visualize the tree structure 
    def draw_tree(self, feature_names=None, class_names=None):
        import matplotlib.pyplot as plt
        
        if self.root is None:
            print("Tree is empty!")
            return

        depth = self._get_depth(self.root)
        width = self._get_width(self.root)
        
        fig = plt.figure(figsize=(max(12, width * 1.5), max(8, depth * 1.5)))
        ax = fig.add_subplot(111)
        ax.axis('off')
        
        self._plot_node(ax, self.root, (0.5, 1.0), (0.5, 1.0), width, depth, feature_names, class_names)
        plt.show()

    def _plot_node(self, ax, node, xy, parent_xy, total_width, total_depth, feature_names, class_names):
        if node.value is not None:
            class_label = node.value
            if class_names:
                class_label = class_names[int(node.value)]
            text = f"Class:\n{class_label}"
            box_color = '#90EE90' if node.value == 1 else '#FFA07A'
        else:
            f_name = f"Feat {node.feature}"
            if feature_names:
                f_name = feature_names[node.feature]
            text = f"{f_name}\n<= {node.threshold:.2f}"
            box_color = '#E0E0E0'

        ax.text(xy[0], xy[1], text, ha='center', va='center', 
                bbox=dict(boxstyle="round,pad=0.5", fc=box_color, ec="black"), 
                fontsize=10)

        if parent_xy != xy:
            ax.plot([parent_xy[0], xy[0]], [parent_xy[1]-0.05, xy[1]+0.05], 'k-', lw=1)

        if node.value is None:
            left_width = self._get_width(node.left)
            right_width = self._get_width(node.right)
            
            next_y = xy[1] - (1.0 / total_depth)
            
            left_x = xy[0] - (0.5 * right_width / total_width / 1.5) 
            right_x = xy[0] + (0.5 * left_width / total_width / 1.5)
            
            offset = 1 / (2 ** ((1/total_depth) + (1-xy[1])*total_depth + 1))
            left_x = xy[0] - offset
            right_x = xy[0] + offset

            self._plot_node(ax, node.left, (left_x, next_y), xy, total_width, total_depth, feature_names, class_names)
            self._plot_node(ax, node.right, (right_x, next_y), xy, total_width, total_depth, feature_names, class_names)

            mid_x_left = (xy[0] + left_x) / 2
            mid_y_left = (xy[1] + next_y) / 2
            ax.text(mid_x_left, mid_y_left, "Yes", fontsize=8, color='blue')
            
            mid_x_right = (xy[0] + right_x) / 2
            mid_y_right = (xy[1] + next_y) / 2
            ax.text(mid_x_right, mid_y_right, "No", fontsize=8, color='red')

    def _get_depth(self, node):
        if not node: return 0
        if node.value is not None: return 1
        return max(self._get_depth(node.left), self._get_depth(node.right)) + 1

    def _get_width(self, node):
        if not node: return 0
        if node.value is not None: return 1
        return self._get_width(node.left) + self._get_width(node.right)

###


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">توضیحات پیاده سازی درخت تصمیم</h3>
    <p>
ساختار درخت تصمیمی که پیاده‌سازی شده بر دو بخش اصلی تکیه دارد. بخش اول برگ‌ها هستند که در آن‌ها مقدار لیبل نهایی برای بردار ویژگی‌ها قرار می‌گیرد. بخش دوم گره‌های میانی هستند که مشخص می‌کنند هر نمونه بر اساس مقدار یک ویژگی خاص باید به زیر درخت چپ برود یا راست. هدف کلی این است که مدل بتواند حتی در عمق‌های کم به بهترین خروجی ممکن برسد و مسیر تصمیم‌گیری تا حد امکان کارا و دقیق باشد.

برای ساخت هر گره از ریشه به سمت برگ‌ها، تلاش می‌کنیم مناسب‌ترین ویژگی و نقطه شکست را برای تقسیم داده‌ها انتخاب کنیم. این کار توسط متد find_best_split انجام می‌شود. در این متد تمام ویژگی‌ها و تمام نقاط شکست ممکن بررسی می‌شوند. دلیل بررسی همه نقاط شکست این است که هیچ تقسیم‌بندی بالقوه‌ای از دست نرود و مدل بتواند بهترین حالت جداکنندگی داده‌ها را پیدا کند. در هر بار امتحان یک threshold جدید، مجموعه داده به دو بخش چپ و راست تقسیم می‌شود و برای هر بخش مقدار آنتروپی محاسبه می‌شود.

آنتروپی نشان می‌دهد که مجموعه چقدر ناخالص است. هرچه احتمال یک لیبل خاص در یک مجموعه بیشتر باشد، مقدار آنتروپی کمتر می‌شود. بنابراین استفاده از آنتروپی به عنوان معیار انتخاب نقطه شکست مناسب است، چون کاهش آنتروپی یعنی داده‌های حاصل از تقسیم، تمیزتر و همگن‌تر شده‌اند و تصمیم‌گیری درباره لیبل آن‌ها قابل اتکاتر است. نکته مهم این است که مقدار آنتروپی هر بخش باید متناسب با تعداد نمونه‌های آن بخش وزن‌دهی شود؛ زیرا ممکن است یک تقسیم آنتروپی کمی تولید کند، اما یکی از طرف‌ها تعداد بسیار کمی نمونه داشته باشد و این کاهش آنتروپی واقعی و قابل اعتماد نباشد.

بعد از انتخاب ویژگی و آستانه مناسب، گره‌های چپ و راست ساخته می‌شوند و فرآیند ساخت درخت به صورت بازگشتی ادامه پیدا می‌کند. این روند تا زمانی ادامه می‌یابد که یکی از شرایط توقف برقرار شود: رسیدن به عمق بیشینه، یکتا شدن لیبل‌ها در یک بخش، یا نبودن ویژگی مناسب برای تقسیم بیشتر. در چنین حالتی، دیگر امکان گسترش درخت وجود ندارد و باید مقدار برگ تعیین شود. مقدار برگ برابر با لیبلی است که در آن بخش بیشترین فراوانی را دارد (رای اکثریت).

در مرحله تست رفتار مدل متفاوت است. در این مرحله دیگر محاسبه آنتروپی انجام نمی‌شود و هدف فقط پیمایش درخت است. مدل از ریشه شروع می‌کند و در هر گره مقدار ویژگی نمونه ورودی را با آستانه‌ای که در مرحله آموزش تعیین شده مقایسه می‌کند. اگر مقدار ویژگی کمتر از threshold باشد، مسیر به سمت چپ می‌رود و در غیر این صورت به سمت راست. این فرآیند تا رسیدن به یک برگ ادامه می‌یابد. مقدار ذخیره‌شده در آن برگ همان برچسب نهایی نمونه تست است.    </p>
</div>


## Runner

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

class Runner:
    def __init__(self, data_path=None, dataframe=None, label_column="Is_finished"):
        self.label_column = label_column
        
        # Check input source: either a file path or an existing dataframe
        if data_path is not None:
            self.df = pd.read_csv(data_path)
        elif dataframe is not None:
            self.df = dataframe.copy()
        else:
            raise ValueError("Error: Provide either 'data_path' or 'dataframe'.")
            
        self.load_data()

    def load_data(self):
        # Work on a copy
        data = self.df.copy()
        
        # Map 'prog_skill' if it exists and is categorical
        if 'prog_skill' in data.columns and data['prog_skill'].dtype == 'object':
            skill_map = {'Low': 0, 'Medium': 1, 'High': 2}
            data['prog_skill'] = data['prog_skill'].map(skill_map)
            # Handle potential missing values with mode
            data['prog_skill'] = data['prog_skill'].fillna(data['prog_skill'].mode()[0])

        # Split features and target
        if self.label_column in data.columns:
            self.y = data[self.label_column]
            self.X = data.drop(self.label_column, axis=1)
        else:
            print(f"Warning: Target column '{self.label_column}' not found!")

    def run(self, test_size=0.3, random_state=42, max_depth=5, verbose=True, show_plot=True):
        # Split train/val
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(
            self.X, self.y, test_size=test_size, random_state=random_state
        )

        # Initialize and train the model
        model = MyDecisionTree(max_depth=max_depth)
        model.fit(self.X_train, self.y_train)
        self.model = model  # Store the trained model

        # Predict
        y_pred = model.predict(self.X_val)
        y_train_pred = model.predict(self.X_train) 

        # Calculate metrics
        metrics = {
            'depth': max_depth,
            'accuracy': accuracy_score(self.y_val, y_pred),
            'precision': precision_score(self.y_val, y_pred, zero_division=0, average='weighted'),
            'recall': recall_score(self.y_val, y_pred, zero_division=0, average='weighted'),
            'f1': f1_score(self.y_val, y_pred, zero_division=0, average='weighted'),
            'train_accuracy': accuracy_score(self.y_train, y_train_pred)
        }

        if verbose:
            print(f"--- Results for Depth: {max_depth} ---")
            print(f"Train Acc: {metrics['train_accuracy']:.4f}")
            print(f"Val Acc:   {metrics['accuracy']:.4f}")
            print(f"Precision: {metrics['precision']:.4f}")
            print(f"Recall:    {metrics['recall']:.4f}")
            print(f"F1 Score:  {metrics['f1']:.4f}")
            print("-" * 30)

        if show_plot:
            cm = confusion_matrix(self.y_val, y_pred)
            disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(self.y))
            disp.plot(cmap=plt.cm.Blues)
            plt.title(f"Confusion Matrix - Depth {max_depth}")
            plt.show()

        return metrics
    
    def find_best_depth(self, min_depth=1, max_depth=15, test_size=0.3):
        print(f"Checking depths from {min_depth} to {max_depth}...")
        results = []
        depths = range(min_depth, max_depth + 1)

        for d in depths:
            # Run without plotting to save time
            res = self.run(max_depth=d, test_size=test_size, show_plot=False, verbose=False)
            results.append(res)

        res_df = pd.DataFrame(results)
        
        # Find best depth based on validation accuracy
        best_row = res_df.loc[res_df['accuracy'].idxmax()]
        print(f"\nFound best depth: {int(best_row['depth'])} -> Val Acc: {best_row['accuracy']:.4f}")

        # Plot Bias-Variance
        plt.figure(figsize=(10, 6))
        plt.plot(res_df['depth'], res_df['train_accuracy'], label='Train Accuracy', marker='o', linestyle='--')
        plt.plot(res_df['depth'], res_df['accuracy'], label='Validation Accuracy', marker='o', linewidth=2)
        plt.plot(res_df['depth'], res_df['f1'], label='Validation F1', marker='s', linewidth=2, color='green')
        plt.axvline(best_row['depth'], color='r', linestyle=':', label=f"Best Depth ({int(best_row['depth'])})")
        
        plt.xlabel('Tree Depth')
        plt.ylabel('Accuracy')
        plt.title('Accuracy vs Depth')
        plt.legend()
        plt.grid(True)
        plt.show()

        return int(best_row['depth'])
    
    def export_tree_structure(self,feature_names=None, class_names=None):
        self.model.draw_tree(feature_names=feature_names, class_names=class_names)

    def predict_submission(self, test_file_path, max_depth, output_file='submission.csv'):
        import os

        self.model = MyDecisionTree(max_depth=max_depth)
        self.model.fit(self.X, self.y)

        output_dir = os.path.dirname(output_file)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)

        df_test = pd.read_csv(test_file_path)
        print(f"Test data loaded. Shape: {df_test.shape}")
        
        if 'prog_skill' in df_test.columns and df_test['prog_skill'].dtype == 'object':
            skill_map = {'Low': 0, 'Medium': 1, 'High': 2}
            df_test['prog_skill'] = df_test['prog_skill'].map(skill_map)
            df_test['prog_skill'] = df_test['prog_skill'].fillna(1)

        X_test = df_test.values if isinstance(df_test, pd.DataFrame) else df_test
        
        y_test_pred = self.model.predict(X_test)
        
        submission = df_test.copy()
        submission[self.label_column] = y_test_pred 
        
        submission.to_csv(output_file, index=False)
        print(f"✅ Submission file saved to: {output_file}")
        
        return submission

###


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">توضیحات کلاس اجرا کننده</h3>
    <p>
کلاس Runner به عنوان چارچوب اصلی اجرای آزمایش‌ها عمل می‌کند و هدف آن این است که تمام مراحل لازم برای بارگذاری داده، انجام پیش‌پردازش، آموزش مدل و ارزیابی خروجی‌ها به صورت یک فرآیند یکپارچه و قابل تکرار مدیریت شود. در زمان ساخته شدن نمونه‌ای از این کلاس، داده‌های ورودی دریافت می‌شوند و عملیات ابتدایی نظیر تبدیل مقادیر دسته‌ای به داده عددی و مدیریت مقادیر گمشده انجام می‌شود تا داده‌ها در قالبی استاندارد و قابل استفاده برای مدل قرار بگیرند.

هسته اصلی این کلاس متدی است که run نام دارد. این متد با دریافت پارامترهایی مانند نسبت داده‌های تست، عمق دلخواه برای درخت و مقدار رندم سید, ابتدا داده‌ها را به مجموعه آموزش و اعتبارسنجی تقسیم می‌کند و سپس مدلی از نوع MyDecisionTree می‌سازد و آن را روی داده‌های آموزشی آموزش می‌دهد. پس از آن، پیش‌بینی روی داده‌های اعتبارسنجی انجام می‌شود و معیارهای ارزیابی نظیر Accuracy، Precision، Recall و F1 محاسبه می‌شوند تا بتوان عملکرد مدل را بر اساس چند جنبه مختلف بررسی کرد. همچنین، ماتریس درهم‌ریختگی رسم می‌شود تا مشخص شود مدل کدام کلاس‌ها را بیشتر اشتباه گرفته و الگوی خطاهای آن چگونه است.

علاوه بر اجرای ساده یک آزمایش، کلاس Runner یک متد دیگر هم برای تنظیم فراپارامتر عمق درخت دارد. این متد که find_best_depth نام‌گذاری شده، عمق‌های مختلف را به صورت پیوسته امتحان می‌کند و برای هر عمق، متد run را فراخوانی می‌کند تا دقت آموزش و اعتبارسنجی ثبت شود. هدف در این مرحله پیدا کردن بهترین عمق ممکن است؛ عمقی که نه آنقدر کوچک باشد که مدل دچار خطای بایاس بالا شود و نه آنقدر بزرگ که مدل روی داده‌های آموزشی overfit شود و روی داده‌های جدید عملکرد ضعیفی داشته باشد. پس از انجام این آزمایش‌ها، نموداری از روند تغییرات دقت آموزش و اعتبارسنجی رسم می‌شود تا نقطه‌ای که مدل بهترین عملکرد را داشته قابل مشاهده باشد.
</div>


<div dir="rtl" style="text-align: justify; line-height: 1.8;">

<h3 style="margin-top: 15px;">تحلیل و معرفی معیارهای ارزیابی</h3>
<p>
در کلاس Runner، برای قضاوت در مورد عملکرد مدل از مجموعه معیارهای استاندارد زیر استفاده شده است. جدول زیر خلاصه فرمول‌ها و کاربرد آن‌ها را نمایش می‌دهد:
</p>

<div align="center">

| معیار (Metric) | فرمول ریاضی | کاربرد اصلی |
| :---: | :---: | :---: |
| **Accuracy** | $\frac{TP + TN}{Total}$ | سنجش دقت کلی (مناسب برای داده‌های متوازن) |
| **Precision** | $\frac{TP}{TP + FP}$ | سنجش اعتمادپذیری پیش‌بینی‌های مثبت |
| **Recall** | $\frac{TP}{TP + FN}$ | سنجش قدرت پوشش‌دهی و یافتن موارد مثبت |
| **F1-Score** | $2 \cdot \frac{P \cdot R}{P + R}$ | میانگین متعادل برای داده‌های نامتوازن |

</div>

<br>

<p>
<b>۱. دقت کلی (Accuracy):</b><br>
این معیار ساده‌ترین شاخص است که نشان می‌دهد مدل در چند درصد مواقع (چه برای پروژه‌های موفق و چه ناموفق) درست عمل کرده است. البته اگر داده‌ها نامتوازن باشند، این معیار کافی نیست.
</p>

<p>
<b>۲. دقت یا صحت (Precision):</b><br>
این معیار روی پاسخ‌های مثبت تمرکز دارد؛ یعنی وقتی مدل پیش‌بینی می‌کند یک پروژه "تمام می‌شود"، چقدر احتمال دارد که این حرف درست باشد؟ این معیار زمانی مهم است که نخواهیم به اشتباه امید واهی بدهیم (False Positive کم باشد).
</p>

<p>
<b>۳. فراخوانی (Recall):</b><br>
این معیار نشان می‌دهد که مدل چقدر "تیزبین" بوده است. یعنی از بین تمام پروژه‌هایی که واقعاً موفق شدند، مدل توانسته چند درصدشان را شناسایی کند؟ این معیار زمانی حیاتی است که نخواهیم هیچ پروژه موفقی را از دست بدهیم (False Negative کم باشد).
</p>

<p>
<b>۴. امتیاز F1-Score:</b><br>
از آنجا که معمولاً بالا بردن Precision باعث افت Recall می‌شود (و برعکس)، معیار F1 به عنوان میانگین هارمونیک این دو، یک عدد واحد و عادلانه برای مقایسه مدل‌ها ارائه می‌دهد.
</p>

<p>
<b>۵. ماتریس درهم‌ریختگی (Confusion Matrix):</b><br>
علاوه بر اعداد بالا، نمودار ماتریس درهم‌ریختگی رسم می‌شود تا دقیقاً ببینیم مدل در کدام کلاس‌ها بیشتر دچار اشتباه شده است (مثلاً آیا بیشتر پروژه‌های موفق را اشتباه تشخیص می‌دهد یا برعکس).
</p>

نکته : هر معیار برای هردو کلاسی که در این داده وجود دارن محاسبه می شه و درنهایت چیزی که به عنوان متریک ذخیره می شه میانگین وزنی آنهاست که وزن آن متناسب با جمعیت هرکدام است

</div>

# Part 1


## Determining the Optimal Tree Depth

In [ ]:
# 1. Initialize runner
runner = Runner(data_path='./part1_dataset/s1_part1_dataset.csv')

# 2. Find best depth
best_depth = runner.find_best_depth(max_depth=10)

###


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">تحلیل تعیین مقدار فراپارامتر عمق بهینه</h3>
    <p>
در انتخاب عمق مناسب برای درخت تصمیم، ما معمولا دقت را مبنای تصمیم‌گیری قرار می‌دهیم و این انتخاب با این فرض است که داده‌ها از نظر توزیع کلاس‌ها متوازن هستند. وقتی داده‌ها متوازن باشند، دقت معیار قابل اعتمادی است، چون مدل نمی‌تواند فقط با تکیه بر یک کلاس غالب امتیاز بالا بگیرد و مجبور است هر دو کلاس را به شکل معنادار یاد بگیرد.

اما اگر داده‌ها نامتوازن باشند، رفتار مدل متفاوت می‌شود. در این حالت ممکن است مدل حتی با عمق خیلی کم، فقط به این دلیل که یک کلاس بیشتر است، تمام نمونه‌ها را به همان کلاس تخصیص دهد و ظاهرا هم دقت خوبی کسب کند، چون بخش زیادی از داده‌ها واقعا متعلق به همان کلاس هستند. این یعنی مدل روی کلاس غالب بایاس شده و عملا هیچ تلاشی برای تشخیص کلاس اقلیت انجام نداده است. اگر عمق را بیشتر هم کنیم، به ظاهر دقت بهتر می‌شود، اما این فقط نتیجه overfit روی کلاس پرجمعیت است و هیچ نشانه‌ای از بهتر شدن واقعی مدل نیست. نمونه‌هایی که واقعا متعلق به کلاس اقلیت هستند همچنان اشتباه پیش‌بینی می‌شوند.

در چنین وضعیتی دقت معیار درستی برای انتخاب عمق نیست و باید از معیاری استفاده شود که حساسیت بیشتری نسبت به خطا در کلاس اقلیت داشته باشد. معیار اف۱ یکی از همین معیارهاست، چون هر دو بخش Precision و Recall را در نظر می‌گیرد و وقتی مدل در تشخیص کلاس کم‌جمعیت ضعیف عمل کند مقدار اف۱ بلافاصله افت می‌کند. به همین دلیل، اف۱ تصویر دقیق‌تری از عملکرد واقعی مدل در داده‌های نامتوازن ارائه می‌دهد.

در حالتی که داده‌ها متوازن باشند، همچنان می‌توان دقت را مبنای انتخاب عمق قرار داد، اما لازم است نمودار اف۱ نیز بررسی شود. اگر روند دقت و اف۱ مشابه باشد، فرض توازن داده‌ها صحیح است و دقت معیار مناسبی باقی می‌ماند. اما اگر اختلاف noticeable بین این دو نمودار دیده شود مشخص می‌شود که فرض اولیه نادرست بوده و باید معیار تصمیم‌گیری را تغییر داد و به سراغ اف۱ رفت.</div>

<div align="center" style="margin-top: 25px;">
    <img src="./Assets/best_depth_1.png" width="720" style="border-radius: 6px; box-shadow: 0 0 8px rgba(0,0,0,0.15);">
    <p style="color: gray; font-size: 0.9em; margin-top: 8px;">
        بررسی روند تغییرات دقت و اف-۱ برای داده‌های تست در عمق‌های مختلف
    </p>
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

 <h3 style="margin-top: 20px; font-weight: 600;">نتیجه و تحلیل نمودار</h3>
 <p>
 بررسی روند دو نمودار دقت و اف۱ نشان می‌دهد که عملکرد مدل روی این دو معیار تقریباً هم‌راستا بوده است و اختلاف قابل‌توجهی میان آن‌ها دیده نمی‌شود. این هماهنگی بیان می‌کند که مدل تحت تأثیر عدم‌توازن برچسب‌ها قرار نگرفته و هیچ‌یک از کلاس‌ها به صورت غیرمنطقی وزن بیشتری در یادگیری نگرفته‌اند. به همین دلیل استفاده از معیار دقت برای انتخاب عمق بهینه معتبر است و مدل در این مسئله نشانه‌ای از بایاس شدن نسبت به کلاس غالب ندارد.
 </p>
 <p>
 همچنین با افزایش عمق، دقت مدل روی داده‌های آموزش به شکل قابل انتظار افزایش پیدا کرده و مدل به‌تدریج تمایل بیشتری به overfit شدن داشته است؛ اما در همین حال، نمودارهای دقت و اف۱ روی داده‌های اعتبارسنجی افت قابل‌توجهی نداشته و رفتار نوسانی نیز از خود نشان نداده‌اند. این ثبات بیان می‌کند که هرچند مدل در عمق‌های بزرگ‌تر بیش از حد با داده‌های آموزش منطبق شده، اما این overfit از نوع مخرب نیست و توانایی مدل در تعمیم‌دهی به داده‌های جدید دچار واگرایی نشده است. این رفتار برای درخت تصمیم طبیعی است و نشان می‌دهد نویز موجود در داده‌ها آن‌قدر زیاد نبوده که افزایش عمق باعث آشفتگی در خروجی مدل شود.
 </p>
 <p>
 در نهایت عمق ۳ به عنوان عمق بهینه انتخاب شده است، زیرا مدل در این نقطه بالاترین دقت را روی داده‌های تست به دست آورده است. عمق‌های کمتر از این مقدار توانایی مدل برای جداسازی داده‌ها را محدود کرده‌اند و مدل در آن ناحیه دچار underfit بوده است. از طرف دیگر، عمق‌های بالاتر از ۳ موجب overfit شدن مدل روی داده‌های آموزش شده‌اند؛ این موضوع در افزایش محسوس دقت train قابل مشاهده است، اما چون عملکرد مدل روی داده‌های تست بهبود پیدا نکرده و حتی کمی کاهش یافته، این ناحیه نشان‌دهنده شروع overfit است. بنابراین عمق ۳ نقطه‌ای است که مدل در آن بیشترین اطلاعات مفید را یاد گرفته و پیچیدگی بیشتر تنها باعث یادگیری نویز شده است.
 </p>
</div>

## Validation Performance at Optimal Depth

In [ ]:
# 3. Run with best depth
runner.run(max_depth=best_depth)

###


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">بررسی و تحلیل خروجی</h3>
</div>

<div align="center" style="margin-top: 25px;">
    <img src="./Assets/validation_result_3.png" width="720" style="border-radius: 6px; box-shadow: 0 0 8px rgba(0,0,0,0.15);">
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
ماتریس درهم‌ریختگی نشان می‌دهد که بخش زیادی از نمونه‌ها در هر دو کلاس به‌درستی تشخیص داده شده‌اند و تراکم قابل توجه روی قطر اصلی بیان می‌کند که مدل در عمق ۳ ساختار اصلی داده را یاد گرفته و دچار Underfit نشده است. همچنین الگوی پیش‌بینی‌ها نشان می‌دهد که مدل خروجی‌ها را به صورت یک‌طرفه و بدون توجه به ویژگی‌ها به یک کلاس خاص اختصاص نداده است. بنابراین اگرچه با توجه به فراوانی بیشتر کلاس ۱ ممکن است تمایل خفیفی به این کلاس وجود داشته باشد، اما تصمیم‌گیری مدل صرفاً بر اساس فراوانی کلاس‌ها نبوده و الگوهای داده به شکل قابل قبولی استخراج شده‌اند.
</p>

<p>
برای تحلیل دقیق‌تر، نرخ خطای هر کلاس بررسی شد. مقدار FN برابر با ۱۹ و مقدار FP برابر با ۲۴ است. این مقادیر نشان می‌دهند که مدل در تشخیص کلاس ۱ عملکرد بهتری داشته و تنها حدود ۹٪ از نمونه‌های واقعی این کلاس را اشتباه تشخیص داده، در حالی که این مقدار برای کلاس ۰ حدود ۲۶٪ است. این اختلاف عملکرد طبیعی است، چون تعداد نمونه‌های کلاس ۱ بیشتر بوده و مدل فرصت بیشتری برای یادگیری ویژگی‌های مرتبط با این کلاس داشته است. با وجود این اختلاف، مدل همچنان بخش عمده‌ای از نمونه‌های کلاس اقلیت را درست تشخیص داده و هیچ نشانه‌ای از یک بایاس شدید و مشکل‌ساز دیده نمی‌شود، بنابراین این میزان بایاس در حدی است که به عملکرد کلی سیستم لطمه نزده است.
</p>

</div>


### چاپ و استخراج درخت


In [ ]:
# Define feature and class names for printing
features = ['group_size', 'meetings', 'skill']
classes = ['Not Finished', 'Finished']

# Print the decision tree structure
runner.export_tree_structure(feature_names=features, class_names=classes)

####


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;"> تصویر ساختار درختی:</h3>
</div>

<div align="center" style="margin-top: 25px;">
    <img src="./Assets/visual_tree_3.png" width="720" style="border-radius: 6px; box-shadow: 0 0 8px rgba(0,0,0,0.15);">
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
با توجه به ساختار درخت، اولین ویژگی‌ای که برای تقسیم‌بندی انتخاب شده prog_skill با آستانه صفر است. قرار گرفتن این ویژگی در ریشه نشان می‌دهد که بیشترین تأثیر را در جداسازی داده‌ها داشته و مدل بیشترین کاهش آنتروپی را از این تقسیم دریافت کرده است. به عبارت دیگر، مهارت اولیه افراد مهم‌ترین عامل در تعیین مسیر تصمیم‌گیری درخت بوده و سایر ویژگی‌ها تنها در لایه‌های بعدی و برای اصلاح یا تکمیل این تقسیم اصلی نقش داشته‌اند.
</p>

</div>


## Validation Performance at first Depth

In [ ]:
runner.run(max_depth=1)

###


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">بررسی و تحلیل خروجی</h3>
</div>

<div align="center" style="margin-top: 25px;">
    <img src="./Assets/validation_result_1.png" width="720" style="border-radius: 6px; box-shadow: 0 0 8px rgba(0,0,0,0.15);">
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
در عمق ۱ مدل مجبور است یک الگوی خیلی کلی بسازد و نمی‌تواند ترکیب چند ویژگی را برای تشخیص بهتر در نظر بگیرد. نتیجه این است که بخش زیادی از نمونه‌ها، مخصوصاً در کلاس ۰، اشتباه پیش‌بینی می‌شوند. بنابراین افت دقت در این حالت رخ داده است، چون مدل امکان ساخت شاخه‌های بیشتر و اصلاح تصمیم‌گیری در مراحل بعد را ندارد.
</p>

<p>
الگوی خطاها نیز نشان می‌دهد که مدل در این عمق به سمت کلاس غالب متمایل شده است. چون تعداد نمونه‌های کلاس ۱ بیشتر است، این رفتار باعث می‌شود نمونه‌های کلاس ۰ با نرخ بسیار بالاتری اشتباه تشخیص داده شوند. این تمایل ناشی از محدودیت پیچیدگی مدل است نه بایاس واقعی در داده
</p>


</div>


# Part 2

## Determining the Optimal Tree Depth

In [ ]:
# 1. Initialize runner
runner = Runner(data_path='./part1_dataset/s1_part2_dataset.csv')

# 2. Find best depth
best_depth = runner.find_best_depth(max_depth=10)

###

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">
 <h3 style="margin-top: 20px; font-weight: 600;">نتیجه و تحلیل نمودار</h3>


<p>
روند کلی نمودار نشان می‌دهد که نقطه بهینه در عمق ۳ اتفاق افتاده است. عمق‌های کمتر از این مقدار نمی‌توانند الگوهای اصلی موجود در داده را به‌درستی تفکیک کنند و مدل در این ناحیه دچار underfit می‌شود، زیرا هنوز شاخه‌های کافی برای جدا کردن رفتارهای مختلف داده تشکیل نشده است.
</p>

<p>
در مقابل، عمق‌های بزرگ‌تر اگرچه دقت مدل روی داده‌های آموزش را افزایش می‌دهند،که نتیجه یادگیری بیش از حد جزئیات و ویژگی‌های غیرضروری داده یا همان overfit است. در چنین حالتی، مدل نسبت به نمونه‌های خاص در داده‌های آموزش حساس می‌شود و همین موضوع باعث می‌شود عملکرد آن روی داده‌های تست نوسان داشته باشد و در برخی عمق‌ها کاهش پیدا کند.
</p>

<p>
بنابراین ناپایداری نمودار در عمق‌های بالا نشان می‌دهد که مدل از یک حد مشخص به بعد اطلاعات مفیدی یاد نمی‌گیرد و فقط به جزئیاتی وابسته می‌شود که برای تعمیم‌دهی مناسب نیستند. مجموع این رفتارها تأیید می‌کند که عمق ۳ بهترین نقطه‌ای است که مدل در آن بیشترین اطلاعات مفید را یاد گرفته و بدون آنکه وارد overfit مخرب شود، عملکرد قابل قبولی روی داده‌های جدید دارد.
</p>


</div>


## Validation Performance at Optimal Depth

In [ ]:
# 3. Run with best depth
runner.run(max_depth=best_depth)

###


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">بررسی و تحلیل خروجی</h3>
</div>

<div align="center" style="margin-top: 25px;">
    <img src="./Assets/validation_part2_3.png" width="720" style="border-radius: 6px; box-shadow: 0 0 8px rgba(0,0,0,0.15);">
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
تراکم بالای درایه‌های روی قطر اصلی نشان می‌دهد که مدل ساختار داده را به‌خوبی یاد گرفته است. علاوه بر این، الگوی پیش‌بینی‌ها نشان می‌دهد که مدل خروجی‌ها را به‌صورت یک‌طرفه به کلاس غالب اختصاص نداده و تصمیم‌گیری آن بر اساس ویژگی‌ها انجام شده است. مقدار FN برابر با ۹ و مقدار FP برابر با ۶ است که نسبت به دیتاست قبلی به‌طور محسوسی کمتر است. این اعداد بیان می‌کنند که مدل در تشخیص هر دو کلاس عملکرد باکیفیتی داشته و تنها تعداد محدودی از نمونه‌ها اشتباه دسته‌بندی شده‌اند.
</p>
‍


</div>


### چاپ و استخراج درخت


In [ ]:
# Define feature and class names for printing
features = ['group_size','meetings_per_week','prog_skill','avg_previous_grade']
classes = ['Not Finished', 'Finished']

# Print the decision tree structure
runner.export_tree_structure(feature_names=features, class_names=classes)

####


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;"> تصویر ساختار درختی:</h3>
</div>

<div align="center" style="margin-top: 25px;">
    <img src="./Assets/visual_tree_part2.png" width="720" style="border-radius: 6px; box-shadow: 0 0 8px rgba(0,0,0,0.15);">
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
با توجه به ساختار درخت، اولین ویژگی‌ای که برای تقسیم‌بندی انتخاب شده avg_previous_grade با آستانه 13.69 است. قرار گرفتن این ویژگی در ریشه نشان می‌دهد که بیشترین تأثیر را در جداسازی داده‌ها داشته و مدل بیشترین کاهش آنتروپی را از این تقسیم دریافت کرده است. به عبارت دیگر، مهارت اولیه افراد مهم‌ترین عامل در تعیین مسیر تصمیم‌گیری درخت بوده و سایر ویژگی‌ها تنها در لایه‌های بعدی و برای اصلاح یا تکمیل این تقسیم اصلی نقش داشته‌اند.
</p>

</div>

## Validation Performance at first Depth

In [ ]:
runner.run(max_depth=1)

###


<div dir="rtl" style="text-align: justify; line-height: 1.8;">
    <h3 style="margin-top: 15px;">بررسی و تحلیل خروجی</h3>
</div>

<div align="center" style="margin-top: 25px;">
    <img src="./Assets/validation_part2_1.png" width="720" style="border-radius: 6px; box-shadow: 0 0 8px rgba(0,0,0,0.15);">
</div>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
در عمق ۱ مدل تنها می‌تواند یک تصمیم کلی بگیرد و امکان بررسی ترکیب چند ویژگی را ندارد. با این حال، خروجی نشان می‌دهد که مدل در این دیتاست حتی در همین عمق کم نیز عملکرد قابل قبولی داشته است. بخش زیادی از نمونه‌ها به‌درستی تشخیص داده شده‌اند و میزان خطا نسبت به حالت مشابه در دیتاست اول بسیار کمتر است. با این وجود، همچنان محدودیت  پیچیدگی مدل باعث شده الگوهای دقیق‌تر داده استخراج نشوند و همین موضوع منجر به کاهش دقت نسبت به عمق‌های بالاتر شده است. الگوی خطاها نشان می‌دهد که مدل کمی به سمت کلاس غالب متمایل شده است، اما این تمایل نسبت به دیتاست اول بسیار کمتر است.
</p>

</div>


# مقایسه نتایج حالت اول و دوم

##

<table style="width: 100%; text-align: center; margin: 25px 0;">
    <tr>
        <td><img src="./Assets/validation_result_1.png" width="800"></td>
        <td><img src="./Assets/validation_result_3.png" width="800"></td>
        <td><img src="./Assets/validation_part2_1.png" width="800"></td>
        <td><img src="./Assets/validation_part2_3.png" width="800"></td>
    </tr>
    <tr>
        <td>دیتاست اول با عمق ۱</td>
        <td>دیتا ست اول با عمق ۳</td>
        <td>دیتاست دوم با عمق ۱</td>
        <td>دیتا ست دوم با عمق ۳</td>‍
    </tr>
</table>

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
با مقایسه ماتریس‌های درهم‌ریختگیِ به‌دست‌آمده از دو دیتاست می‌توان مشاهده کرد که مدل در دیتاست دوم، چه در عمق ۱ و چه در عمق ۳، عملکرد به مراتب بهتری داشته است. حتی با پیچیدگی بسیار کم و تنها یک نقطه تصمیم، مدل توانسته بخش بزرگی از نمونه‌ها را به درستی تشخیص دهد، در حالی که همین شرایط در دیتاست اول منجر به خطاهای قابل توجه شده بود. این تفاوت نشان می‌دهد که فیچر جدیدی که در ریشه قرار گرفته، قدرت تفکیک بالایی دارد و توانسته مرزبندی مؤثری میان کلاس‌ها ایجاد کند؛ موضوعی که در دیتاست اول به دلیل نبود چنین فیچری محقق نشده بود.
</p>


</div>


# Generating Submission File

In [ ]:
import os
# learn_source = "./part1_dataset/s1_part1_dataset.csv"
learn_source = "./part1_dataset/s1_part2_dataset.csv"

# test_source = "s1_part1_test.csv"
test_source = "s1_part2_test.csv"

# 1. Initialize runner
runner = Runner(data_path=learn_source)

# 2. Find best depth
best_depth = runner.find_best_depth(max_depth=10)

# 3. Train and Predict on test set and save submission
part_name = test_source.split('/')[-1].split('_')[1]

output_path = f"part1_submissions/submission_{part_name}.csv"
runner.predict_submission(test_file_path=test_source, max_depth=best_depth, output_file=output_path)

# سوالات

<div dir="rtl" style="text-align: justify; line-height: 1.9; font-size: 1.05em;">

<p>
اضافه شدن این فیچر جدید باعث شد مدل از همان سطح اول بتواند یک مرزبندی واضح بین کلاس‌ها ایجاد کند. در واقع این فیچر آن‌قدر اطلاعات مفید در خودش دارد که رابطه‌اش با برچسب خروجی تقریباً مستقیم است، و همین موضوع باعث شده مدل بدون اینکه مجبور باشد شاخه‌های زیادی بسازد، بخش بزرگی از داده‌ها را درست تشخیص دهد. به همین دلیل حتی یک درخت ساده و کم‌عمق هم توانسته به نتیجه خوبی برسد؛ در حالی که قبل از اضافه شدن این فیچر، مدل باید چند سطح تقسیم پشت‌سر هم ایجاد می‌کرد تا تازه بتواند به یک ساختار قابل‌استفاده برسد.
</p>

<p>
این تجربه خیلی روشن نشان می‌دهد که داشتن یک فیچر خوب گاهی حتی از پیچیده‌تر کردن مدل هم مهم‌تر است. وقتی فیچرهای موجود قدرت تفکیک کافی نداشته باشند، مدل به‌ناچار پیچیده می‌شود تا بتواند الگوهای داده را پیدا کند؛ اما وقتی فیچر مناسبی به داده اضافه شود، مدل همان اطلاعات را با عمق و پیچیدگی کمتر یاد می‌گیرد. به همین خاطر است که ساخت فیچر درست می‌تواند نیاز به عمق زیاد، تعداد زیاد تقسیم‌ها یا حتی مدل‌های پیچیده‌تر را کم کند. در نهایت این موضوع نشان می‌دهد که کیفیت داده در خیلی از مواقع از پیچیده بودن مدل مهم‌تر است.
</p>

<p>
اگر این فیچر اضافه نمی‌شد و فقط عمق درخت را زیاد می‌کردیم، مدل مجبور می‌شد تعداد زیادی تقسیم انجام دهد تا نبود این اطلاعات را جبران کند. این کار احتمالاً دقت آموزش را بالا می‌برد اما مدل کم‌کم وارد حالت overfit می‌شد و روی داده‌های جدید عملکرد خوبی نداشت. در چنین وضعیتی مدل به‌جای یادگیری الگوی اصلی، به جزئیات اتفاقی و نویز داده وابسته می‌شد. بنابراین افزایش عمق بدون داشتن یک فیچر مناسب، فقط پیچیدگی مدل را بیشتر می‌کرد بدون اینکه واقعاً کیفیت پیش‌بینی بهتر شود.
</p>

</div>
‍